In [1]:
import argparse

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from model_weight_dropout import ResNet
import numpy as np
import torchvision.datasets as dset
import torchvision.transforms as T
import os
import random
import copy

In [2]:
def check_accuracy(model, loader, gpu_dtype):
    num_correct = 0
    num_samples = 0
    model.eval()
    for X, y in loader:

        X_var = Variable(X.type(gpu_dtype), requires_grad=True)

        scores = model(X_var)
        _, preds = scores.data.cpu().max(1) 

        num_correct += (preds == y).sum()
        num_samples += preds.size(0)

    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [61]:
model = ResNet(n=5, res_option='A', use_dropout=True)
model = torch.load('model weights prune 40')
model.eval()

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (layers1): Sequential(
    (0): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout2d(p=0.67, inplace=True)
      (relu2): ReLU(inplace=True)
    )
    (1): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), str

In [4]:
def exponential_discretisation(X, bits):
    std_W = np.std(X)
    max_W = np.fabs(X).max()
    x0 = (std_W / max_W) * 0.5**(bits-1)
    signs = np.sign(X)
    q_a = np.fabs(X.flatten()) / max_W
    n = 2 ** (bits - 1)
    q = 1 / x0 ** (1 / (n - 1))
    assert (x0 < 1)
    xx = [0] + [x0 * q ** i for i in range(n)]
    q_mean = np.zeros(q_a.size, dtype=np.float) + np.mean(q_a[q_a > xx[n - 1]])
    for i in range(len(xx) - 2):
        ii = np.logical_and(xx[i] < q_a, q_a <= xx[i + 1])
        q_mean[ii] = np.mean(q_a[ii])
    new_W = np.reshape(q_mean, X.shape) * signs * max_W
    return new_W, np.corrcoef(np.ravel(X), np.ravel(new_W))[1,0], np.array(xx) * max_W

In [29]:
def exp_quant(model, B=8):
    with torch.no_grad():
        model1 = copy.deepcopy(model)
        model1 = model1.cpu()
        i = 0
        for name, param in model1.named_parameters():
            if len(param.shape) != 1:
                q_weights = param.detach().numpy()

                for j in range(q_weights.shape[0]):
                    if i != 0 and i != len(list(model.named_parameters())) - 1:
                        q_weights[j, ] = exponential_discretisation(q_weights[j,], B)[0]

                    else:
                        q_weights[j,] = exponential_discretisation(q_weights[j,], 8)[0]
                param.data = torch.tensor(q_weights)
            i += 1
    return model1

In [24]:
gpu_dtype = torch.cuda.FloatTensor
transform_augment = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomCrop(32, padding=4)])
transform_normalize = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
NUM_TRAIN = 45000
NUM_VAL = 5000

cifar10_test = dset.CIFAR10('./dataset', train=False, download=True,
                            transform=transform_normalize)

loader_test = DataLoader(cifar10_test, batch_size=128)

Files already downloaded and verified


In [62]:
model1 = exp_quant(model, 3)
model1 = model1.type(gpu_dtype)
check_accuracy(model1, loader_test, gpu_dtype)

Got 3765 / 10000 correct (37.65)


In [63]:
model1 = exp_quant(model, 4)
model1 = model1.type(gpu_dtype)
check_accuracy(model1, loader_test, gpu_dtype)

Got 7191 / 10000 correct (71.91)


In [64]:
model1 = exp_quant(model, 5)
model1 = model1.type(gpu_dtype)
check_accuracy(model1, loader_test, gpu_dtype)

Got 8355 / 10000 correct (83.55)


In [65]:
model1 = exp_quant(model, 6)
model1 = model1.type(gpu_dtype)
check_accuracy(model1, loader_test, gpu_dtype)

Got 8511 / 10000 correct (85.11)


In [66]:
model1 = exp_quant(model, 7)
model1 = model1.type(gpu_dtype)
check_accuracy(model1, loader_test, gpu_dtype)

Got 8538 / 10000 correct (85.38)


In [67]:
model1 = exp_quant(model, 8)
model1 = model1.type(gpu_dtype)
check_accuracy(model1, loader_test, gpu_dtype)

Got 8526 / 10000 correct (85.26)


In [68]:
model= model.type(gpu_dtype)
check_accuracy(model, loader_test, gpu_dtype)

Got 8527 / 10000 correct (85.27)
